In [54]:
import glob
fnames = glob.glob('../data/processed/cd1/02_말뭉치/현대/*/*/형태분석_말뭉치/*.txt')
len(fnames)

479

In [13]:
import sys
sys.path.append('../py')
from utils import load_processed_corpus, tageojeol_to_tuple
from hangle import decompose, compose, moum_begin, moum_end, jaum_begin, jaum_end, kor_begin, kor_end

주어진 (어절, 단어열, 태그열)을 이용하여 L={명사,동사,형용사,어근}, R={조사,어미}로 나뉘어지도록 어절을 나누는 함수입니다. 

### L=명사가 되는 경우

1. 명사 + 조사: 사람/N + 은/J
1. 명사\* + 조사: 사람/N + 사이/N + 에/J
1. 명사 + 명사파생접미사 + 조사: 인간/N + 성/XSN + 이/J
1. 명사 + [동사/형용사]파생접미사 + 어미: 사용되는 = 사용/N + 되/XSV + 는/ETM
1. 어근 + [동사/형용사]파생접미사 + 어미: 간단한 = 간단/XR + 하/XSA + ᆫ/ETM

되다/하다 동사는 앞의 명사를 취하여 동사/형용사화 할 수 있지만, 이는 동사가 아닌 명사로 취급한다. 어근(XR) 역시 명사로 쓰일 수 있기 때문에 명사로 취급한다. (되다/하다 동사는 명사를 나타내는 매우 좋은 feature이다)

### L=동사/형용사가 되는 경우

1. 동사 + 어미: 갔다 = 가/VV + ㅆ다/E

### L=동사/형용사로 인정하지 않는 경우

1. 보조용언 + 어미: 있다고 = 있/VX + 다고/EC

        있/VX 127462
        하/VX 84812
        않/VX 70039
        지/VX 42071
        주/VX 33671
        보/VX 30586
        ...


In [41]:
def is_all_complete_hangle(eojeol):
    for char in eojeol:
        if not (kor_begin <= ord(char) <= kor_end):
            return False
    return True

def to_lr(e, w, t):
    i = 0
    for i_, ti in enumerate(t):
        if not (ti[0] == 'N' or ti == 'XSN' or ti[:2] == 'VV' or ti[:2] == 'VA' or ti == 'XR'):
            break
        i = i_
    lw = e[:len(''.join(w[:i+1]))]
    r = e[len(lw):]
    
    # 아빤 = 아빠/N + ㄴ/J
    # 갈꺼야 = 가/V + ㄹ/E + 꺼야/E
    if (t[i][0] == 'N' or t[i][0] == 'V') and (jaum_begin <= ord(w[i+1][0]) <= jaum_end):
        last_l = decompose(lw[-1])
        l0 = lw[:-1] + compose(last_l[0], last_l[1], ' ')
        return lw, r, t[0][0], l0

    # 가? = 가/V + ㅏ/E + ?/S
    # 먹었어 = 먹/V + 었어/E
    return lw, r, t[0][0].replace('X','N'), ''.join(w[:i+1])

def is_compound_noun(t):
    if len(t) <= 1:
        return False
    n_count = len([ti for ti in t if ti[0] == 'N' or ti == 'XSN'])
    if len(t) == n_count:
        return True
    if n_count <= 1:
        return False
    if t[0][0] == 'N' and [-1][0] == 'N':
        return True

In [57]:
def print_tolr(args):
    print('L=%s, R=%s, tag=%s, L(원형)=%s' % args)

print_tolr(to_lr(e='타고',   w=['타','고'],       t=['VV','EC']))
print_tolr(to_lr(e='가는데?', w=['가','는데','?'], t=['VV','EF','SF']))
print_tolr(to_lr(e='갈꺼야',  w=['가','ㄹ','꺼야'], t=['VV','E','E']))
print_tolr(to_lr(e='인간성이', w=['인간','성','이'], t=['NNG','XSN','JE']))
print_tolr(to_lr(e='사용되는', w=['사용','되','는'], t=['NNG','XSV','ETM']))
print_tolr(to_lr(e='간단한',  w=['간단','하','ㄴ'], t=['XR','XSA','ETM']))

L=타, R=고, tag=V, L(원형)=타
L=가, R=는데?, tag=V, L(원형)=가
L=갈, R=꺼야, tag=V, L(원형)=가
L=인간성, R=이, tag=N, L(원형)=인간성
L=사용, R=되는, tag=N, L(원형)=사용
L=간단, R=한, tag=N, L(원형)=간단


In [52]:
with open('../data/processed/lr/lrdb.csv', 'w', encoding='utf-8') as f:
    f.write('%s\n' % '\t'.join(['eojeol', 'l', 'r', 'tag', 'lstemmed']))
    for fname in fnames:
        texts, texttags = load_processed_corpus(fname)
        for text, tag in zip(texts, texttags):
            for eojeol, eojeoltag in zip(text.split(), tag.split()):
                if not is_all_complete_hangle(eojeol):
                    continue
                    
                try:
                    w, t = tageojeol_to_tuple(eojeoltag)
                except:
                    continue
                
                if len(t) <= 1:
                    continue
                if (t[0][0] != 'N' and t[0][0] != 'V'):
                    continue
                if is_compound_noun(t):
                    continue

                try:
                    l, r, y, l0 = to_lr(eojeol, w, t)
                    f.write('%s\n' % '\t'.join([eojeol, l, r, y, l0]))
                except:
                    continue